# 自動微分
:label:`sec_autograd`
 
正如 :numref:`sec_calculus`中所說，求導是幾乎所有深度學習優化算法的關鍵步驟。
雖然求導的計算很簡單，只需要一些基本的微積分。
但對於複雜的模型，手工進行更新是一件很痛苦的事情（而且經常容易出錯）。
 
深度學習框架通過自動計算導數，即*自動微分*（automatic differentiation）來加快求導。
實際中，根據設計好的模型，系統會構建一個*計算圖*（computational graph），
來追蹤計算是哪些數據通過哪些操作組合起來產生輸出。
自動微分使系統能夠隨後反向傳播梯度。
這裡，*反向傳播*（backpropagate）意味著追蹤整個計算圖，填充關於每個參數的偏導數。

## 一個簡單的例子
 
作為一個演示例子，(**假設我們想對函數$y=2\mathbf{x}^{\top}\mathbf{x}$關於列向量$\mathbf{x}$求導**)。
首先，我們創建變量`x`並為其分配一個初始值。


In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**在我們計算$y$關於$\mathbf{x}$的梯度之前，需要一個地方來儲存梯度。**]
重要的是，我們不會在每次對一個參數求導時都分配新的記憶體。
因為我們經常會成千上萬次地更新相同的參數，每次都分配新的記憶體可能很快就會將記憶體耗盡。
注意，一個純量函數關於向量$\mathbf{x}$的梯度是向量，並且與$\mathbf{x}$具有相同的形狀。


In [2]:
x.requires_grad_(True)  # 等價於x=torch.arange(4.0,requires_grad=True)
x.grad  # 默認值是None

(**現在計算$y$。**)


In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

`x`是一個長度為4的向量，計算`x`和`x`的點積，得到了我們賦值給`y`的標量輸出。
接下來，[**通過調用反向傳播函數來自動計算`y`關於`x`每個分量的梯度**]，並打印這些梯度。


In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函數$y=2\mathbf{x}^{\top}\mathbf{x}$關於$\mathbf{x}$的梯度應為$4\mathbf{x}$。
讓我們快速驗證這個梯度是否計算正確。


In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

[**現在計算`x`的另一個函數。**]


In [6]:
# 在預設情況下，PyTorch會累積梯度，我們需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非純量變量的反向傳播
 
當`y`不是純量時，向量`y`關於向量`x`的導數的最自然解釋是一個矩陣。
對於高階和高維的`y`和`x`，求導的結果可以是一個高階張量。
 
然而，雖然這些更奇特的物件確實出現在高級機器學習中（包括[**深度學習中**]），
但當調用向量的反向計算時，我們通常會試圖計算一批訓練樣本中每個組成部分的損失函數的導數。
這裡(**，我們的目的不是計算微分矩陣，而是單獨計算批量中每個樣本的偏導數之和。**)


In [7]:
# 對非純量調用backward需要傳入一個gradient參數，該參數指定微分函數關於self的梯度。
# 本例只想求偏導數的和，所以傳遞一個1的梯度是合適的
x.grad.zero_()
y = x * x
# 等價於y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 分離計算
 
有時，我們希望[**將某些計算移動到記錄的計算圖之外**]。
例如，假設`y`是作為`x`的函數計算的，而`z`則是作為`y`和`x`的函數計算的。
想像一下，我們想計算`z`關於`x`的梯度，但由於某種原因，希望將`y`視為一個常數，
並且只考慮到`x`在`y`被計算後發揮的作用。
 
這裡可以分離`y`來返回一個新變量`u`，該變量與`y`具有相同的值，
但丟棄計算圖中如何計算`y`的任何信息。
換句話說，梯度不會向後流經`u`到`x`。
因此，下面的反向傳播函數計算`z=u*x`關於`x`的偏導數，同時將`u`作為常數處理，
而不是`z=x*x*x`關於`x`的偏導數。


In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由於記錄了`y`的計算結果，我們可以隨後在`y`上調用反向傳播，
得到`y=x*x`關於的`x`的導數，即`2*x`。


In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度計算
 
使用自動微分的一個好處是：
[**即使構建函數的計算圖需要通過Python控制流（例如，條件、循環或任意函數調用），我們仍然可以計算得到的變量的梯度**]。
在下面的代碼中，`while`循環的迭代次數和`if`語句的結果都取決於輸入`a`的值。


In [10]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

讓我們計算梯度。


In [11]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

我們現在可以分析上面定義的`f`函數。
請注意，它在其輸入`a`中是分段線性的。
換言之，對於任何`a`，存在某個常量標量`k`，使得`f(a)=k*a`，其中`k`的值取決於輸入`a`，因此可以用`d/a`驗證梯度是否正確。


In [12]:
a.grad == d / a

tensor(False)

## 小結
 
* 深度學習框架可以自動計算導數：我們首先將梯度附加到想要對其計算偏導數的變量上，然後記錄目標值的計算，執行它的反向傳播函數，並訪問得到的梯度。

## 練習
 
1. 為什麼計算二階導數比一階導數的開銷要更大？
1. 在運行反向傳播函數之後，立即再次運行它，看看會發生什麼。
1. 在控制流的例子中，我們計算`d`關於`a`的導數，如果將變量`a`更改為隨機向量或矩陣，會發生什麼？
1. 重新設計一個求控制流梯度的例子，運行並分析結果。
1. 使$f(x)=\sin(x)$，繪製$f(x)$和$\frac{df(x)}{dx}$的圖像，其中後者不使用$f'(x)=\cos(x)$。


[Discussions](https://discuss.d2l.ai/t/1759)


回答(不一定正確):

1. 為什麼計算二階導數比一階導數的開銷要更大？
---

計算二階導數比一階導數的開銷更大，主要是由以下原因導致的：

1. **額外的計算依賴性**：
   - 一階導數僅需要計算函數對輸入的偏導數。
   - 二階導數則需要計算一階導數對輸入的導數，因此需要追蹤和存儲更多的計算圖。
   
2. **計算圖的大小增加**：
   - 自動微分中，計算圖會表示所有變數之間的依賴性。計算二階導數時，圖的深度和複雜性增加，需要更多內存和計算資源。

3. **額外的反向傳播步驟**：
   - 計算一階導數需要執行一次反向傳播。而計算二階導數需要在一階導數的基礎上再次進行反向傳播。

4. **數值穩定性**：
   - 高階導數的計算更容易受到數值誤差的影響，可能需要更多精確的浮點運算來避免不穩定的結果。

計算二階導數涉及更多的計算依賴性、更大的計算圖和額外的反向傳播步驟，因此比計算一階導數的開銷更大。



---
2. 在運行反向傳播函數之後，立即再次運行它，看看會發生什麼。
---

在 PyTorch 中執行反向傳播（`backward()`）後，計算圖默認會被釋放，因為 PyTorch 的計算圖僅用於一次性反向傳播。這是為了節省內存空間。

因此，嘗試在執行完一次 `backward()` 後再次執行，會引發錯誤：
```
RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed.
```

要避免這種情況，您可以使用 `retain_graph=True` 在第一次反向傳播時保留計算圖，以便之後可以再次使用。

---

### **程式碼範例：**
這段程式碼模擬問題，並解決它。

```python
import torch

# 建立需要梯度的張量
x = torch.tensor([2.0, 3.0], requires_grad=True)
y = x.pow(2).sum()  # y = x1^2 + x2^2

# 第一次反向傳播
y.backward(retain_graph=True)  # 保留計算圖
print("第一次反向傳播後的梯度：", x.grad)

# 嘗試再次反向傳播
try:
    y.backward()  # 再次反向傳播（若未保留計算圖，會報錯）
    print("第二次反向傳播後的梯度：", x.grad)
except RuntimeError as e:
    print("再次反向傳播時的錯誤：", e)
```

---

### **分析結果：**
1. **第一次反向傳播**：
   - 會計算梯度並存儲在 `x.grad` 中。
2. **第二次反向傳播**：
   - 如果未使用 `retain_graph=True`，將會引發錯誤。
   - 若使用 `retain_graph=True`，可以成功執行，但需注意內存開銷。

---
3. 在控制流的例子中，我們計算`d`關於`a`的導數，如果將變量`a`更改為隨機向量或矩陣，會發生什麼？
---
1. **梯度的重新計算**：
   - 如果將變量 `a` 更改為隨機向量或矩陣，計算 `d` 關於 `a` 的導數時，自動微分工具（如 PyTorch）將會重新構建計算圖並計算梯度。梯度的結果會根據 `a` 的新值而變化。

2. **動態計算圖特性**：
   - PyTorch 的動態計算圖允許對輸入值的靈活更改。當 `a` 被更改為隨機向量或矩陣後，新的計算會基於新值生成對應的計算圖，並計算梯度。

3. **結果的數學意義**：
   - 對於隨機輸入，梯度的具體數值會因隨機輸入的值不同而不同，但導數的計算方式（基於函數的偏導）仍然一致。

---

### **程式碼範例：**

以下是一段 PyTorch 程式碼來測試這個情境：

```python
import torch

# 定義一個控制流函數
def control_flow_example(a):
    b = a * 2 if a.sum() > 1 else a / 2
    return b.sum()

# 初始化變量 a，並啟用梯度
a = torch.tensor([0.5, 1.0, -0.5], requires_grad=True)

# 計算控制流函數的輸出
d = control_flow_example(a)
d.backward()  # 計算 d 關於 a 的導數
print("原始梯度：", a.grad)

# 更改變量 a 為隨機向量，重新計算
a = torch.randn(3, requires_grad=True)
d = control_flow_example(a)
d.backward()
print("隨機向量的新梯度：", a.grad)

# 更改變量 a 為隨機矩陣，重新計算
a = torch.randn(3, 3, requires_grad=True)
d = control_flow_example(a)
d.backward()
print("隨機矩陣的新梯度：", a.grad)
```

---

### **分析結果：**
1. **梯度值的變化**：
   - 由於 `a` 的值改變，`control_flow_example` 中的條件分支可能會選擇不同的計算路徑，因此導致梯度值的變化。
2. **計算圖的重新構建**：
   - 每次變更 `a` 的值後，PyTorch 會根據新值重新構建計算圖，確保正確計算梯度。

---
4. 重新設計一個求控制流梯度的例子，運行並分析結果。
---

### 回答：

1. **控制流梯度的特性**：
   - 控制流梯度的計算會根據輸入數值的不同而觸發不同的計算路徑。梯度值會反映選擇的計算路徑，因此它可能是分段的。

2. **重新設計的例子**：
   - 設計一個包含條件分支的函數，條件基於輸入值。
   - 計算梯度，並展示梯度如何根據控制流的不同選擇而改變。

3. **分析結果**：
   - 不同的控制流路徑會影響梯度值。
   - 梯度反映了當前輸入值所觸發的計算路徑的偏導。

---

### **程式碼範例**：

```python
import torch

# 定義控制流函數
def custom_control_flow(x):
    # 根據條件選擇不同計算路徑
    if x.sum() > 0:
        y = x ** 2  # 條件 1: 平方
    else:
        y = x.abs()  # 條件 2: 絕對值
    return y.sum()

# 測試 1: x 為正向量
x = torch.tensor([1.0, 2.0, -3.0], requires_grad=True)
output = custom_control_flow(x)
output.backward()
print("x 為正向量時的梯度：", x.grad)

# 測試 2: x 為負向量
x = torch.tensor([-1.0, -2.0, -3.0], requires_grad=True)
output = custom_control_flow(x)
output.backward()
print("x 為負向量時的梯度：", x.grad)

# 測試 3: x 為隨機值
x = torch.randn(3, requires_grad=True)
output = custom_control_flow(x)
output.backward()
print("x 為隨機值時的梯度：", x.grad)
```

---

### **分析結果**：

1. **條件選擇影響梯度**：
   - 當 `x.sum() > 0` 時，梯度反映的是平方運算的導數（`2*x`）。
   - 當 `x.sum() <= 0` 時，梯度反映的是絕對值的導數（`1` 或 `-1`，取決於符號）。

2. **控制流的動態性**：
   - 計算圖動態調整，以匹配當前輸入值觸發的分支路徑。

3. **重新設計的目的**：
   - 這例子展示了控制流的條件如何影響梯度計算的路徑和結果，幫助理解動態計算圖的優勢。

---
5. 使$f(x)=\sin(x)$，繪製$f(x)$和$\frac{df(x)}{dx}$的圖像，其中後者不使用$f'(x)=\cos(x)$。
---

### 回答：

1. **計算 \( \frac{df(x)}{dx} \) 的方法**：
   - 使用自動微分框架（如 PyTorch）計算梯度，而不是直接使用 \( f'(x) = \cos(x) \) 的公式。
   - 梯度通過 `torch.autograd` 自動計算。

2. **繪製圖像**：
   - 使用 Matplotlib 繪製 \( f(x) = \sin(x) \) 和 \( \frac{df(x)}{dx} \)。
   - 梯度的數值會來自於自動微分計算。

3. **結果分析**：
   - \( \sin(x) \) 的導數應該與 \( \cos(x) \) 的圖像一致，但計算過程依賴於自動微分而非顯式公式。

---

### **程式碼範例**：

```python
import torch
import matplotlib.pyplot as plt
import numpy as np

# 定義 x 範圍
x_vals = np.linspace(-2 * np.pi, 2 * np.pi, 100)
x = torch.tensor(x_vals, requires_grad=True)

# 定義 f(x) = sin(x)
y = torch.sin(x)

# 計算梯度
y.backward(torch.ones_like(x))  # 對 y 求梯度
grad = x.grad  # 取得梯度值

# 繪製圖像
plt.figure(figsize=(10, 6))

# 繪製 f(x)
plt.plot(x_vals, np.sin(x_vals), label="f(x) = sin(x)")

# 繪製 df(x)/dx
plt.plot(x_vals, grad.detach().numpy(), label="df(x)/dx (calculated via autograd)")

# 標註
plt.title("f(x) = sin(x) and its derivative df(x)/dx")
plt.xlabel("x")
plt.ylabel("y")
plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
plt.axvline(0, color='gray', linewidth=0.5, linestyle='--')
plt.legend()
plt.grid()

plt.show()
```

---

### **結果分析**：

1. **圖像的對比**：
   - \( f(x) = \sin(x) \) 為正弦曲線。
   - \( \frac{df(x)}{dx} \) 的自動微分結果應與 \( \cos(x) \) 完全一致。

2. **不使用顯式公式**：
   - 梯度的計算純粹依賴於自動微分，驗證了框架的正確性和動態計算圖的強大功能。

